This file is originally from [wsgisler/ibm-ponder-challenge on GitHub](https://github.com/wsgisler/ibm-ponder-challenge/blob/master/2020-03/Ponder%20This%20-%20March%202020%20-%20Challenge.ipynb)

---

# IBM Ponder This - March 2020 - Challenge

by Walter Sebastian Gisler

## Problem Statement

The original problem statement can be found here: https://www.research.ibm.com/haifa/ponderthis/challenges/March2020.html

In the classic 3x3 tic-tac-toe game, either player can win or at least force a draw if he plays correctly.
On a 3x4 board, the first player can always win if she plays correctly, but if the players are working together, it is possible to get a draw.

This month we define a new game (dic-dac-doe) played on a 10x10 board by three players (X, O, and +), where the first to get at least three consecutive squares with their sign (horizontally, vertically or diagonally) - wins.
Three peace-loving players are playing and trying together to get to a draw, but after 80 moves, an intensely competitive win-only demon takes over all of their minds and controls their remaining 20 moves, trying to get one of them to win. Can they play the first 80 moves in such a way that even the demon won't be able to prevent a draw?

Supply your answer as the status of the game after the 80th move as an 10x10 array of 'X', 'O', '+', and '.', where the first three symbols represent the moves of the players and the dots represent empty squares.

Bonus '*' for a solution that lets the demon act no later than the 186th move on a 15x15 board.

In [13]:
%pip install -q docplex cplex

Note: you may need to restart the kernel to use updated packages.


# Solution

This problem can be solved easily using a MIP approach. I'll use Cplex as a MIP solver. Let's first define our model, our players and the rows and columns of the board:

In [14]:
from docplex.mp.model import Model

model = Model('ponderthis')

players = [0,1,2]
player_symbol = ['x','o','+']
board_size = 10
rows = range(1,board_size+1)
cols = range(1,board_size+1)

Next, we will define the variables. For each field on the board we will have 3 binary variables. One for each player: If this variable is 1, it means that this field contains the players symbol.

In [15]:
choice = {(r,c,p): model.binary_var() for r in rows for c in cols for p in players}

We will also define a variable for each field that can be used to determine whether the field is empty:

In [16]:
empty = {(r,c): model.binary_var() for r in rows for c in cols}

Next, we need to add some rules. First of all, a field can contain at most one symbol:

In [17]:
for r in rows:
    for c in cols:
        model.add(model.sum(choice[(r,c,p)]for p in players) <= 1)

We want to find out what the solution looks like after 80 moves. Hence, we have to define that the players made 27 (player 1) or 26 (player 2 and 3) moves:

In [18]:
for p in players:
    model.add(model.sum(choice[(r,c,p)] for r in rows for c in cols) == (27 if p <= 1 else 26))

If a field does not contain the symbol of one of the players, it must be empty:

In [19]:
for r in rows:
    for c in cols:
        model.add(model.sum(choice[(r,c,p)] for p in players) + empty[(r,c)] == 1)

If we would solve this model now, we would get solutions after 80 moves, but these solutions could already contain a winner and they certainly wouldn't guarantee that none of the players can win the game. We therefore need to add conditions to specify that no 3 horizontal fields, no 3 consecutive fields in a column and no 3 consecutive fields on a diagonal can contain the same symbol:

In [20]:
# no player should have 3 horizontal symbols
for p in players:
    for r in rows:
        for c in cols[:-2]:
            model.add(choice[(r,c,p)]+choice[(r,c+1,p)]+choice[(r,c+2,p)] <= 2)
            model.add(choice[(r,c,p)]+choice[(r,c+1,p)]+choice[(r,c+2,p)]+empty[(r,c)]+empty[(r,c+1)]+empty[(r,c+2)] <= 2)
        
# no player should have 3 vertical symbols
for p in players:
    for r in rows[:-2]:
        for c in cols:
            model.add(choice[(r,c,p)]+choice[(r+1,c,p)]+choice[(r+2,c,p)] <= 2)
            model.add(choice[(r,c,p)]+choice[(r+1,c,p)]+choice[(r+2,c,p)]+empty[(r,c)]+empty[(r+1,c)]+empty[(r+2,c)] <= 2)
        
# no player should have diagonal symbols (top left -> bottom right)
for p in players:
    for r in rows:
        for c in cols:
            if r+2 <= board_size and c+2 <= board_size:
                model.add(choice[(r,c,p)]+choice[(r+1,c+1,p)]+choice[(r+2,c+2,p)] <= 2)
                model.add(choice[(r,c,p)]+choice[(r+1,c+1,p)]+choice[(r+2,c+2,p)]+empty[(r,c)]+empty[(r+1,c+1)]+empty[(r+2,c+2)] <= 2)
            
# no player should have diagonal symbols (bottom left -> top right)
for p in players:
    for r in rows:
        for c in cols:
            if r+2 <= board_size and c-2 >= 1:
                model.add(choice[(r,c,p)]+choice[(r+1,c-1,p)]+choice[(r+2,c-2,p)] <= 2)
                model.add(choice[(r,c,p)]+choice[(r+1,c-1,p)]+choice[(r+2,c-2,p)]+empty[(r,c)]+empty[(r+1,c-1)]+empty[(r+2,c-2)] <= 2)

Explanation: for each player, we look at every possible 3-row, 3-column and 3-diagonal. We then add two rules. The first one says that the solution with our 80 moves (defined by the choice variable) can not contain the symbol of this particular player more than twice. The second condition defines that the number of empty fields in this 3-row / 3-column / 3-diagonal together with the number of symbols belonging to this player can not be larger than 2. Theoretically, the first condition is included in the second one but I think it pays off to add both to get tighter bounds. We are now ready to solve this model.

In [21]:
model.solve(log_output = True)

Using size restricted mode (Could not find directory for cpxchecklic).
CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.


DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 400 vars, 1931 consts, CPLEX code=1016

A solution is found really quickly. Let's take a look at it:

In [ ]:
for r in rows:
    row_string = ''
    for c in cols:
        sym = '.'
        for p in players:
            if choice[(r,c,p)].solution_value > 0.5:
                sym = player_symbol[p]+''
        row_string += sym
    print(row_string)

DOcplexException: Model<ponderthis> did not solve successfully

We can easily verify that this solution indeed doesn't allow us to win the game for any of the players. Even if all the dots (empty fields after 80 moves) are replaced with any of the players symbols, it is not possible to get a 3-row / 3-column / 3-diagonal with three identical symbols.

# The Larger Problem

That was simple, so let's try to find a solution for a 15x15 board and 186 moves. I will first define a method that will allow me to prioritize certain variables for the branching, we will use this later:

In [ ]:
def set_branching_priority(dvars, weights, brdirs = []):
    # set priority ordering on a collection of variables,
    # given a list of weights, and a list of -1,0,1 directions
    ldvars = list(dvars)
    lweights = list(weights)
    if brdirs == []: # If the branching direction is not given, set it automatically
        brdirs = [0 for var in dvars]
    ldirs = list(brdirs)
    if ldvars:
        m = ldvars[0].model
        cpx = m.get_cplex()
        cpx.order.set([(dv.index, w, brd) for dv, w, brd in zip(ldvars, lweights, ldirs)])

I will now make the 80-move code more general, add it into its own function and make some adjustments to improve the performance.

In [ ]:
def find_general_solution(board_size, moves):

    model = Model('ponderthis')

    players = [0,1,2]
    player_symbol = ['x','o','+']
    rows = range(1,board_size+1)
    cols = range(1,board_size+1)
    objective = []

    choice = {(r,c,p): model.binary_var() for r in rows for c in cols for p in players}
    empty = {(r,c): model.binary_var() for r in rows for c in cols}

    # per square, there is exactly  one symbol:
    for r in rows:
        for c in cols:
            model.add(model.sum(choice[(r,c,p)]for p in players) <= 1)

    # number of symbols per player:
    for p in players:
        model.add(model.sum(choice[(r,c,p)] for r in rows for c in cols) == (moves//3 + (1 if moves%3 <= p+1 else 0)))
        
    # empty squares
    for r in rows:
        for c in cols:
            model.add(model.sum(choice[(r,c,p)] for p in players) + empty[(r,c)] == 1)
    
    # no player should have 3 horizontal symbols
    for p in players:
        for r in rows:
            for c in cols[:-2]:
                viol = model.continuous_var()
                objective.append(viol)
                model.add(choice[(r,c,p)]+choice[(r,c+1,p)]+choice[(r,c+2,p)] <= 2+viol)
                model.add(choice[(r,c,p)]+choice[(r,c+1,p)]+choice[(r,c+2,p)]+empty[(r,c)]+empty[(r,c+1)]+empty[(r,c+2)] <= 2+viol)
            
    # no player should have 3 vertical symbols
    for p in players:
        for r in rows[:-2]:
            for c in cols:
                viol = model.continuous_var()
                objective.append(viol)
                model.add(choice[(r,c,p)]+choice[(r+1,c,p)]+choice[(r+2,c,p)] <= 2+viol)
                model.add(choice[(r,c,p)]+choice[(r+1,c,p)]+choice[(r+2,c,p)]+empty[(r,c)]+empty[(r+1,c)]+empty[(r+2,c)] <= 2+viol)
            
    # no player should have diagonal symbols (top left -> bottom right)
    for p in players:
        for r in rows:
            for c in cols:
                if r+2 <= board_size and c+2 <= board_size:
                    viol = model.continuous_var()
                    objective.append(viol)
                    model.add(choice[(r,c,p)]+choice[(r+1,c+1,p)]+choice[(r+2,c+2,p)] <= 2+viol)
                    model.add(choice[(r,c,p)]+choice[(r+1,c+1,p)]+choice[(r+2,c+2,p)]+empty[(r,c)]+empty[(r+1,c+1)]+empty[(r+2,c+2)] <= 2+viol)
                
    # no player should have diagonal symbols (bottom left -> top right)
    for p in players:
        for r in rows:
            for c in cols:
                if r+2 <= board_size and c-2 >= 1:
                    viol = model.continuous_var()
                    objective.append(viol)
                    model.add(choice[(r,c,p)]+choice[(r+1,c-1,p)]+choice[(r+2,c-2,p)] <= 2+viol)
                    model.add(choice[(r,c,p)]+choice[(r+1,c-1,p)]+choice[(r+2,c-2,p)]+empty[(r,c)]+empty[(r+1,c-1)]+empty[(r+2,c-2)] <= 2+viol)
    
    set_branching_priority(empty.values(), [10 for i in empty])
    model.minimize(model.sum(objective))
    model.solve(log_output = True)

    for r in rows:
        row_string = ''
        for c in cols:
            sym = '.'
            for p in players:
                if choice[(r,c,p)].solution_value > 0.5:
                    sym = player_symbol[p]
            row_string += sym
        print(row_string)

Notice two changes:

- First of all, we are setting a higher branching priority for the empty variables. That means that we want that these variables are set first. This is not necessary, but it improves the performance significantly
- Second, our contains to avoid 3 consecutive fields containing the same symbol now has a slack variable "viol" and we are minimizing the total sum of all slack variables. Obviously, in a valid solution this sum would be equal to 0 since a non-zero slack variable means that the constraint is violated. This is another great trick to improve performance

I first tested the model without these tricks, but it took really long to find a solution. That's why I decided to try some common MIP tricks like these and found that these work really well, even for larger instances.

Now, let's give this a try:

In [ ]:
find_general_solution(15, 185)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Tried aggregator 1 time.
MIP Presolve eliminated 225 rows and 0 columns.
Reduced MIP has 4596 rows, 3084 columns, and 25599 nonzeros.
Reduced MIP has 900 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (14.23 ticks)
Found incumbent of value 8736.000000 after 0.03 sec. (24.58 ticks)
Probing time = 0.00 sec. (0.81 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 2184 rows and 0 columns.
Reduced MIP has 2412 rows, 3084 columns, and 16863 nonzeros.
Reduced MIP has 900 binaries, 2184 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.04 sec. (24.40 ticks)
Probing time = 0.00 sec. (0.55 ticks)
Clique table members: 225.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation s

  58146 42718        0.0000   263        4.0000        0.0000  1946818  100.00%
Elapsed time = 85.06 sec. (50262.51 ticks, tree = 846.68 MB, solutions = 37)
  58824 43043        3.0000   296        4.0000        0.0000  1959035  100.00%
  60545 44402        3.0000   305        4.0000        0.0000  1996597  100.00%
  62677 46517        2.0000   333        4.0000        0.0000  2068206  100.00%
  64883 48623        1.0000   282        4.0000        0.0000  2137260  100.00%
  66993 50193        cutoff              4.0000        0.0000  2188686  100.00%
  69164 52763        0.0000   326        4.0000        0.0000  2277132  100.00%
  71239 54143        cutoff              4.0000        0.0000  2323104  100.00%
  73108 55673        3.0000   324        4.0000        0.0000  2377954  100.00%
  75124 57773        2.0000   328        4.0000        0.0000  2447722  100.00%
  77101 59795        2.0000   336        4.0000        0.0000  2519597  100.00%
Elapsed time = 100.55 sec. (59804.79 ticks,

 416010 365231        3.0000   341        4.0000        0.0000 14137285  100.00%
Elapsed time = 498.23 sec. (253513.93 ticks, tree = 7842.42 MB, solutions = 37)
Nodefile size = 5787.81 MB (4769.41 MB after compression)
 423242 372048        2.0000   363        4.0000        0.0000 14396787  100.00%
 430150 378300        0.0000   341        4.0000        0.0000 14636785  100.00%
 437355 384598        1.0000   326        4.0000        0.0000 14871256  100.00%
 444523 390494        2.0000   335        4.0000        0.0000 15094547  100.00%
 448445 395570        3.0000   335        4.0000        0.0000 15294430  100.00%
 453966 400367        2.0000   319        4.0000        0.0000 15484390  100.00%
 460655 405785        0.0000   331        4.0000        0.0000 15698675  100.00%
 467597 411254        2.0000   349        4.0000        0.0000 15911726  100.00%
 474054 416809        3.0000   337        4.0000        0.0000 16139753  100.00%
 481179 423899        1.0000   312        4.0000    

This is obviously not as fast as for the 10x10 board, but we are still able to find a solution within a reasonable amount of time. I was running this on a 32 core machine and it took about 10 minutes to find a solution.

Thanks for another fun challenge, IBM! Looking forward to seeing other people's solution approaches. As always, MIP is a quick solution and is implemented in a couple of minutes but there might be a much more suitable algorithm that is faster overall.

An example for a 15x15 solution looks as follows:

In [ ]:
solution = "\
O.XOX.OXO.X+X.O\
++OO++X.+O+.X+.\
.X+X.OXOXX+OOX+\
O+.XO++X.OX++XO\
XOO+X.O++X.OX+.\
+X+.XOX.OXO+OXX\
.XOO++OO++X.O++\
X+X+.X+X.OXO+.O\
+.XOO+.XO++XOO+\
OO+.XOO+X.O++X.\
.X+X+O+.XOX.O+X\
XXOO+XOO++OOX.O\
O++XOX.+OX+X+O+\
.X+.O+X+.XOX+X.\
+.XOX.+OXO.+X.O"